# Imports

In [1]:
import sys
import pandas as pd
import os
import zipfile

import joblib
import pandas as pd

sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..','..','..')))

from src.utils.model import load_models_auto
from src.utils.analysis import create_predict_function, create_quantile_function
from src.utils.model import load_models_auto

ALPHA = 0.1
SPLIT = "both"
NUMBER_OF_WEEK = 4
USE_AUTO_SCAN = True  # Toggle this to switch between the loading of the last model of the manual load of a specific model
FINAL_MODEL = "qrf"
MODEL_DIR = "../../../models/"
EVAL_DIR = "../../../data/evaluation/"
INPUT_DIR = "../../../data/input/"

# Loading of the data / models

Loading of the inference data.

In [2]:
# load the dataset
inference_data = pd.read_csv(f"{EVAL_DIR}dataset_baseline.csv")
inference_data = inference_data.set_index("ObsDate")


Loading of the final models.

In [3]:
# Load models based on conditions
final_models = []
if FINAL_MODEL == "mapie":
    if USE_AUTO_SCAN:
        final_models = load_models_auto("mapie_quantile", f"{MODEL_DIR}final/")
    else:
        final_models.append(joblib.load(f"{MODEL_DIR}final/mapie_quantile_2025-01-17_15-15-04_week0.pkl"))
        final_models.append(joblib.load(f"{MODEL_DIR}final/mapie_quantile_2025-01-17_15-15-11_week1.pkl"))
        final_models.append(joblib.load(f"{MODEL_DIR}final/mapie_quantile_2025-01-17_15-15-17_week2.pkl"))
        final_models.append(joblib.load(f"{MODEL_DIR}final/mapie_quantile_2025-01-17_15-15-17_week3.pkl"))
elif FINAL_MODEL == "qrf":

    if USE_AUTO_SCAN:
        final_models = load_models_auto("qrf_quantile", f"{MODEL_DIR}final/")
    else:
        final_models.append(joblib.load(f"{MODEL_DIR}final/qrf_quantile_2025-01-17_15-15-04_week0.pkl"))
        final_models.append(joblib.load(f"{MODEL_DIR}final/qrf_quantile_2025-01-17_15-15-11_week1.pkl"))
        final_models.append(joblib.load(f"{MODEL_DIR}final/qrf_quantile_2025-01-17_15-15-17_week2.pkl"))
        final_models.append(joblib.load(f"{MODEL_DIR}final/qrf_quantile_2025-01-17_15-15-17_week3.pkl"))
elif FINAL_MODEL == "ebm":
    if USE_AUTO_SCAN:
        final_models = load_models_auto("ebm_ensemble", f"{MODEL_DIR}final/")
    else:
        final_models.append(joblib.load(f"{MODEL_DIR}final/EBM_ensemble_2025-01-17_15-15-04_week0.pkl"))
        final_models.append(joblib.load(f"{MODEL_DIR}final/EBM_ensemble_2025-01-17_15-15-11_week1.pkl"))
        final_models.append(joblib.load(f"{MODEL_DIR}final/EBM_ensemble_2025-01-17_15-15-17_week2.pkl"))
        final_models.append(joblib.load(f"{MODEL_DIR}final/EBM_ensemble_2025-01-17_15-15-17_week3.pkl"))


In [4]:
predictions = inference_data[['station_code']].copy()
y_pred_test_quantile = {}
y_pred_test = {}
X_test = inference_data.drop(columns=['station_code'])
for i in range(NUMBER_OF_WEEK):
    
    if FINAL_MODEL == "qrf":
        # reorder the columns
        X_test = X_test[final_models[0].feature_names_in_]
    predict_adjusted = create_predict_function(final_models, i, FINAL_MODEL)
    quantile_adjusted = create_quantile_function(final_models, i, FINAL_MODEL, ALPHA)
    
    y_pred_test[i] = predict_adjusted(X_test)
    y_pred_test_quantile[i] = quantile_adjusted(X_test)

for i in range(NUMBER_OF_WEEK):
    predictions[f"week_{i}_pred"] = y_pred_test[i]
    predictions[f"week_{i}_sup"] = y_pred_test_quantile[i][:,1]
    predictions[f"week_{i}_inf"] = y_pred_test_quantile[i][:,0]


# Saving of the predictions

In [5]:
# save the predictions to a csv file
predictions["ObsDate"] = X_test.index
predictions.to_csv(f"{EVAL_DIR}predictions.csv", index=False)

In [6]:
# Create a ZIP file containing predictions.csv
with zipfile.ZipFile(f"{EVAL_DIR}predictions.zip", 'w', zipfile.ZIP_DEFLATED) as zipf:
    zipf.write(f"{EVAL_DIR}predictions.csv", "predictions.csv")


You are ready to submit go to codabench and submit the zip file that have been generated in My Submissions > Phase 1.

![Alt text](CodaBench.png)
